# 第三方库导入

In [2]:
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.model_selection import cross_val_score, ShuffleSplit, train_test_split  
from sklearn.datasets import make_classification  # 用于生成模拟数据  
from sklearn.base import clone  
from sklearn.metrics import accuracy_score  
from tqdm import tqdm  
from sklearn.preprocessing import StandardScaler    
from sklearn.svm import SVC   
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
from sklearn.gaussian_process import GaussianProcessClassifier  
import seaborn as sns 
from sklearn.model_selection import train_test_split 
import seaborn as sns 
from sklearn import metrics 
import plotly.express as px 
from sklearn.pipeline import Pipeline 
from sklearn.multiclass import OneVsRestClassifier
#models 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import BernoulliNB 
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.neural_network import MLPClassifier 
from sklearn.linear_model import SGDClassifier 
from xgboost import XGBClassifier 
from catboost import CatBoostClassifier 
import xgboost as xgb 
import catboost 
from sklearn.model_selection import train_test_split 
#scoring 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score, precision_score, recall_score, average_precision_score, roc_auc_score, precision_recall_curve, roc_curve, auc 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import label_binarize,LabelEncoder
import shap
import re
from scipy.interpolate import make_interp_spline

d:\ProgramData\Anaconda3\envs\paddle_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 临床特征指标数据读取
datacln = pd.read_excel('data all NEW(1).xlsx',header=0,sheet_name='clinical data')
clinic = datacln.iloc[:,2:]
clinic = clinic.iloc[:,:-1]

In [4]:
clinic.rename(columns={"Lab ID":"Sample"},inplace=True)
clinic.head(5)

,Sample,Class,Absti,SV,PH,C,TSC,SM,RC,SR,...,NP,IM,VCL,VSL,VAP,LIN,STR,WOB,ALH,BCF
0,SS0145,Terato,2,1.5,7.3,59.8,89.8,67.2,3,80,...,8.1,32.8,103.6,55.0,70.9,53.1,77.5,68.5,2.6,13.5
1,SS0135,Terato,4,3.5,7.4,61.4,214.7,71.4,2,80,...,14.8,28.6,86.8,38.3,56.7,44.1,67.6,65.3,2.6,11.9
2,SS0130,Terato,3,1.5,7.5,53.8,80.7,62.7,3,80,...,11.2,37.3,95.9,56.0,69.7,58.4,80.4,72.7,2.3,12.8
3,SS0128,Terato,5,4.0,7.4,62.4,249.5,63.8,3,70,...,16.8,36.2,82.7,39.1,53.6,47.3,73.0,64.8,2.6,11.2
4,SS0126,Terato,3,4.0,7.5,65.2,260.9,79.3,3,90,...,10.4,20.7,108.2,59.0,76.0,54.5,77.6,70.3,2.7,13.6


In [5]:
label_encoder = LabelEncoder()  
clinic['Class']= label_encoder.fit_transform(clinic['Class']) 
label_encoder.classes_

array(['Astheno', 'AsthenoTerato', 'Normal', 'OAT', 'Terato'],
      dtype=object)

In [6]:
X = clinic.loc[:,['C','N%','SM','PR','IM']]
y = clinic['Class']

In [7]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42, stratify=y)

In [8]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# XGBoost模型参数
params_xgb = {
    'learning_rate': 0.02,            # 学习率，控制每一步的步长，用于防止过拟合。典型值范围：0.01 - 0.1
    'booster': 'gbtree',              # 提升方法，这里使用梯度提升树（Gradient Boosting Tree）
    'objective': 'multi:sofmax',   # 损失函数，这里使用逻辑回归，用于二分类任务
    'max_leaves': 127,                # 每棵树的叶子节点数量，控制模型复杂度。较大值可以提高模型复杂度但可能导致过拟合
    'verbosity': 1,                   # 控制 XGBoost 输出信息的详细程度，0表示无输出，1表示输出进度信息
    'seed': 42,                       # 随机种子，用于重现模型的结果
    'nthread': -1,                    # 并行运算的线程数量，-1表示使用所有可用的CPU核心
    'colsample_bytree': 0.6,          # 每棵树随机选择的特征比例，用于增加模型的泛化能力
    'subsample': 0.7,                 # 每次迭代时随机选择的样本比例，用于增加模型的泛化能力
    'eval_metric': 'logloss'          # 评价指标，这里使用对数损失（logloss）
}


# 初始化XGBoost分类模型
model_xgb = xgb.XGBClassifier(**params_xgb)


# 定义参数网格，用于网格搜索
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],  # 树的数量
    'max_depth': [3, 4, 5, 6, 7],               # 树的深度
    'learning_rate': [0.01, 0.02, 0.05, 0.1],   # 学习率
}


# 使用GridSearchCV进行网格搜索和k折交叉验证
grid_search = GridSearchCV(
    estimator=model_xgb,
    param_grid=param_grid,
    scoring='neg_log_loss',  # 评价指标为负对数损失
    cv=5,                    # 5折交叉验证
    n_jobs=-1,               # 并行计算
    verbose=1                # 输出详细进度信息
)

# 训练模型
grid_search.fit(X_train, y_train)

# 输出最优参数
print("Best parameters found: ", grid_search.best_params_)
print("Best Log Loss score: ", -grid_search.best_score_)

# 使用最优参数训练模型
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters found:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 400}
Best Log Loss score:  0.17482129168479982


In [9]:

from sklearn.metrics import classification_report
# 预测测试集
y_pred = best_model.predict(X_test)
# 输出模型报告， 查看评价指标
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.75      0.75      0.75         4
           2       1.00      1.00      1.00         5
           3       0.80      1.00      0.89         4
           4       1.00      1.00      1.00         4

    accuracy                           0.89        19
   macro avg       0.91      0.85      0.86        19
weighted avg       0.91      0.89      0.89        19



In [10]:
confusion = confusion_matrix(y_test, y_pred) 
plt.figure(figsize=(20,20),dpi=500)
fig = px.imshow(confusion, labels=dict(x="Predicted Value", y="Actual Vlaue"), x=[0,1,2,3,4],y=[0,1,2,3,4],text_auto=True, title='Confusion Matrix',) 
fig.update_layout(title_x=0.5) 
fig.show()

<Figure size 10000x10000 with 0 Axes>

In [11]:
import joblib  
# 保存模型  
joblib.dump(best_model, 'XGBoost.pkl')  
  
print("模型已保存为 'XGBoost.pkl'")

模型已保存为 'XGBoost.pkl'


# Streamlit 程序开发

In [12]:
! pip install streamlit

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
      --------------------------------------- 0.2/8.7 MB 5.9 MB/s eta 0:00:02
     -- ------------------------------------- 0.5/8.7 MB 6.7 MB/s eta 0:00:02
     ------ --------------------------------- 1.5/8.7 MB 10.7 MB/s eta 0:00:01
     --------------- ------------------------ 3.4/8.7 MB 18.1 MB/s eta 0:00:01
     ----------------------- ---------------- 5.2/8.7 MB 23.7 MB/s eta 0:00:01
     ------------------------ --------------- 5.4/8.7 MB 20.1 MB/s eta 0:00:01
     ------------------------- -------------- 5.6/8.7 MB 17.8 MB/s eta 0:00:01
     -------------------------- ------------- 5.8/8.7 MB 16.0 MB/s eta 0:00:01
     --------------------------- ------------ 6.0/8.7 MB 14.7 MB/s eta 0:00:01
     ---------------------------- ----------- 6.2/8.7 MB 13.6 MB/s eta 0:00:01
     ----------------------------- ---------- 6.3/8.7 MB 12.7 MB/s eta 0:00:01


In [15]:
import streamlit as st
import joblib
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

# Load the model
model = joblib.load('XGBoost.pkl')

# Define feature options

# Define feature names
feature_names = ['C','N%','SM','PR','IM']

# Streamlit user interface
st.title(" Semen quality test Predictor")

# 精液浓度: numerical input
C = st.number_input("C:", min_value=1, max_value=300, value=50)

# N: categorical selection
N = st.number_input("N%:", min_value=1, max_value=10, value=3)

# SM: categorical selection
SM =st.number_input("SM:", min_value=10, max_value=100, value=50)

# PR: numerical input
PR = st.number_input("PR:", min_value=1, max_value=150, value=30)

# IM: numerical input
IM = st.number_input("IM:", min_value=0, max_value=100, value=50)


# Process inputs and make predictions
feature_values = [C, N, SM, PR, IM]
features = np.array([feature_values])

if st.button("Predict"):  
    # Predict class and probabilities  
    predicted_class = model.predict(features)[0]  
    predicted_proba = model.predict_proba(features)[0]  
  
    # Display prediction results  
    class_labels = ['Astheno', 'AsthenoTerato', 'Normal', 'OAT', 'Terato'] # 假设的类别标签  
    predicted_class_label = class_labels[predicted_class]  
    st.write(f"**Predicted Class**: {predicted_class_label}")  
    st.write(f"**Prediction Probabilities**: {predicted_proba}")  
  
    # Generate advice based on prediction results  
    probability = predicted_proba[predicted_class] * 100  
  
    # 根据不同的类别生成不同的建议  
    advice = ""  
    if predicted_class == 0:  
        advice = (  
            f"根据我们的模型，您属于{predicted_class_label}。模型预测您属于此类的概率为{probability:.1f}%。"  
            "具体建议需要根据该类别的含义来确定。"  
        )  
    elif predicted_class == 1:  
        advice = (  
            f"根据我们的模型，您可能面临较高的风险（属于{predicted_class_label}）。"  
            f"模型预测您属于此类的概率为{probability:.1f}%。"  
            "建议您立即咨询相关专家以获取进一步评估和治疗建议。"  
        )  
    elif predicted_class == 2:  
        advice = (  
            f"根据我们的模型，您属于{predicted_class_label}的概率较高，为{probability:.1f}%。"  
            "请继续关注您的健康状况，并根据需要寻求医疗建议。"  
        )  
    elif predicted_class == 3:  
        advice = (  
            f"模型预测您属于{predicted_class_label}的概率为{probability:.1f}%。"  
            "请注意保持健康的生活方式，并根据需要咨询医生。"  
        )  
    elif predicted_class == 4:  
        advice = (  
            f"模型预测您属于{predicted_class_label}的概率为{probability:.1f}%。"  
            "具体的行动建议需要根据该类别的具体情况来确定。"  
        )  
    else:  
        advice = "无法识别的类别，请检查模型输出。"  
  
    st.write(advice)

    # Calculate SHAP values and display force plot
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(pd.DataFrame([feature_values], columns=feature_names))

    shap.force_plot(explainer.expected_value[predicted_class], shap_values[:,:,predicted_class], pd.DataFrame([feature_values], columns=feature_names), matplotlib=True)
    plt.savefig("shap_force_plot.png", bbox_inches='tight', dpi=1200)

    st.image("shap_force_plot.png")

2024-08-31 21:01:13.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-31 21:01:13.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-31 21:01:13.902 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-31 21:01:13.902 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-31 21:01:13.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-31 21:01:13.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-31 21:01:13.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-31 21:01:13.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar